In [ ]:
pip install -q yahoo_fin

In [ ]:
pip install --upgrade yfinance

In [3]:
mkdir -p data results log

In [4]:
from google.colab import drive
drive.mount('/content/drive')
# ! find /content/drive/MyDrive/colab/ -type f -mtime +180 -delete -print


Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd

In [6]:
def show_prediction(name, suffix):
  df = pd.read_csv(f'results/{name}_{suffix}_predictions.csv')
  df['predicted_return'] = ((df['predicted_max']+df['predicted_min']+df['predicted_close'])/3 - df['current_price'])/df['current_price']
  return df.sort_values('predicted_return', ascending=False, key=abs)


In [7]:
def run_command_with_check(command):
  """Runs a command, prints stderr, and raises an exception if it fails."""
  process = subprocess.run(command, shell=True, capture_output=True, text=True, check=False)

  if process.returncode != 0:
    print("stderr:", process.stderr)  # Print stderr only on error
    raise subprocess.CalledProcessError(process.returncode, process.args, output=process.stdout, stderr=process.stderr)
  else:
    # Print or process stdout if needed
    #print("stdout:", process.stdout)
    pass

  return process.returncode


In [8]:
import os
import subprocess
def run_model_nn_cmd(prefix, version, do_train=False, window_size=None, future_days=None, hourly=False, copy_tickers=True):
    # copy drive/MyDrive/colab/results/nnreturn.py to local folder
    tickers = f"{prefix}tickers.csv"
    run_command_with_check(f"cp  /content/drive/MyDrive/colab/nnreturn.py .")
    if copy_tickers:
      run_command_with_check(f"cp  /content/drive/MyDrive/colab/tickers/{tickers} .")
    if not do_train:
      run_command_with_check(f"cp  /content/drive/MyDrive/colab/models/{prefix}_{version}_*.pth .")
      # Run nnreturn.py in results folder and throw exception if error
      today = pd.to_datetime('today').strftime('%m%d')
      run_command_with_check(f"rename 's/{version}/{today}/' *.pth")
      cmd_arg = ""
      if window_size:
        cmd_arg += f" --window-size {window_size}"
      if future_days:
        cmd_arg += f" --future-days {future_days}"
      if hourly:
        cmd_arg += " --hourly"
      run_command_with_check(f"python3 nnreturn.py --tickers-file {prefix}tickers.csv --mode analyze {cmd_arg}")
      run_command_with_check(f"cp {prefix}*predictions.csv results")
      # Get today's date

      return show_prediction(prefix, today)
    else:
      raise Exception("Not implemented")


In [ ]:
run_model_nn_cmd('etf', '0730').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
1,ARKK,82.47,2026-01-12,2026-02-09,72.86,83.58,79.55,-0.046158
2,ARKW,151.45,2026-01-12,2026-02-09,136.57,155.30,145.04,-0.038385
0,ARKF,49.30,2026-01-12,2026-02-09,45.40,50.01,47.03,-0.036917
31,XME,120.53,2026-01-12,2026-02-09,112.64,121.25,116.05,-0.032219
29,XLV,157.02,2026-01-12,2026-02-09,148.08,160.34,152.95,-0.020571
10,IXP,122.11,2026-01-12,2026-02-09,118.29,123.62,116.99,-0.020282
28,XLU,42.47,2026-01-12,2026-02-09,41.91,43.56,44.32,0.018680
14,JETS,29.30,2026-01-12,2026-02-09,27.92,29.59,28.80,-0.018089
22,XLC,117.50,2026-01-12,2026-02-09,112.50,121.22,113.65,-0.014553
4,DXYZ,30.35,2026-01-12,2026-02-09,28.38,32.37,29.00,-0.014278


In [ ]:
# 0716 is hourly 20
run_model_nn('idx', '0716', window_size=20, future_days=8, hourly=True).head(10)

In [ ]:
import runml.pipeline
from runml import pipeline,findata
#from importlib import reload
# reload(runml.pipeline)

In [ ]:
# Skip some days from consecutive window to lessen test /train overlap
# findata.G_NON_OVERLAP= 0

Run following if like to save results in drive

In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    # Simplify following using apply
    def calculate_alloc(row):
        gain_low = min(row['Gain_l'], 0)
        gain_high = max(row['Gain_h'], 0)
        gain = min(row['Gain'], gain_high)
        average_gain = (gain_low+gain_high + 2*gain)/4
        if (average_gain == 0):
          return 0
        avg_accuracy = (row['Accu']+row['Accu_l']+row['Accu_h'])/3
        alloc =  avg_accuracy/stop_loss - (1-avg_accuracy)*profit_factor/abs(average_gain)
        return round(max(alloc, 0), 2)
    df['Alloc'] = df.apply(calculate_alloc, axis=1)
    return df

pd.options.display.max_columns = None

In [ ]:
import os
# findata.EPOCHS=200
def run_model(prefix, version, tickers, do_train):
  if (not do_train):
    os.system(f"cp  /content/drive/MyDrive/colab/results/{prefix}-{version}-* ./results/")
  mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(),
                         pipeline.AddMA(200), pipeline.Adj()]))
  df= pipeline.runModelCombinedVola(tickers, f"{prefix}-{version}", mod, do_train)
  # Save datafrme to prefix-version-prediction.csv
  df.to_csv(f"results/{prefix}-{version}-prediction.csv")
  return df



In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax' # Change to minmax
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'BYRN', 'CELH', 'CFLT', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ELF', 'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HCP', 'HIMS', 'HOOD', 'HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JAKK', 'JKS', 'KD', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR', 'MRVL', 'MXL','MTCH', 'MU',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PAR', 'PANW',  'PAYX', 'PD', 'PI', 'PINS',
           'RBLX', 'RKLB', 'RMBS', 'RUM', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'SOUN', 'TEM', 'TEAM', 'TDOC', 'TMDX', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
df = run_model('ipos', '2a', tickers, True)

In [ ]:
addAllocHL(df, 0.10, 1)
df[(df.Gain>0)].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'DXYZ',
            'EEM', 'FDN', 'FPX',
            'ICLN', 'IGV', 'IJK', 'IJR', 'IPO',
            'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'JXI', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'URA', 'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
df1 = run_model('etf', '1b', tickers1, True)



In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

In [ ]:
df1.to_csv('results/etf_12b_predictions.csv')

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax'
tickers3 = [ 'AIRI', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS',  'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CHGG', 'CLOV', 'CRCT', 'CXM',
            'DLO', 'DM', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MITK', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PLUG', 'PTON', 'PUBM',
            'REAL', 'REI', 'RIG', 'RVYL',
            'SABR', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'VTEX', 'VSAT', 'YEXT']
df3 = run_model('vols', '12b', tickers3, False)

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'KO', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
df4 = run_model('mcap', '1b', tickers4, True)

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CX', 'DBOEY', 'DNNGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FDRVF', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']

df5 = run_model('gstock', '2a', tickers5, False)

In [ ]:
addAllocHL(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.G_SCALER = 'standard'
tickers6 = ['ARGT', 'EWD', 'FLN',
           'ECH', 'EPHE', 'EWA', 'EWG', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'KWT', 'THD', 'TUR', 'VNM', 'XCEM']
df6 = run_model('etf', '12a', tickers6, False)

In [ ]:
addAllocHL(df6, 0.07, 1)
df6.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'standard'
tickers7 = ['AAL', 'ADI', 'ALB', 'AMR','ANF', 'APO', 'AQN', 'ARCH', 'ARE', 'ASC',
            'BAH', 'BAX', 'BKR', 'BLK', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IFF', 'IP', 'IPI', 'IVZ',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW', 'ZTS']
df7 = run_model('val', '1a', tickers7, False)

In [ ]:
addAllocHL(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
tickers8 = [ 'DBC', 'FXB', 'FXE', 'FXF', 'FXY',
            'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

df8 =  run_model('com', '12a', tickers8, True)

In [ ]:
addAllocHL(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

In [ ]:
dfp = run_model_nn_cmd('pennystock', '1226')

In [ ]:
dfp[dfp['predicted_return'] > 0].head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
6,ATOM,2.30,2026-01-14,2026-02-11,2.11,3.54,2.75,0.217391
29,GOGO,4.82,2026-01-14,2026-02-11,4.70,6.86,5.99,0.213693
70,SLE,0.66,2026-01-14,2026-02-11,0.53,0.89,0.83,0.136364
42,LFMD,3.90,2026-01-14,2026-02-11,3.71,4.42,5.16,0.135897
49,NNOX,3.03,2026-01-14,2026-02-11,2.39,4.09,3.53,0.101210
10,BKKT,20.96,2026-01-14,2026-02-11,18.92,27.81,22.27,0.097328
39,INVZ,1.08,2026-01-14,2026-02-11,0.97,1.28,1.29,0.092593
5,AUR,4.54,2026-01-14,2026-02-11,4.23,6.27,4.36,0.091043
13,BULL,8.28,2026-01-14,2026-02-11,7.27,9.84,9.70,0.079308
11,BGSF,5.47,2026-01-14,2026-02-11,5.11,6.42,6.08,0.073126


In [ ]:
run_model_nn_cmd('bluechipstock', '0221').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
11,CRM,227.11,2026-01-16,2026-02-13,228.85,253.66,243.11,0.065005
15,DELL,120.53,2026-01-16,2026-02-13,117.43,129.76,130.61,0.044830
12,COST,963.61,2026-01-16,2026-02-13,937.25,1081.84,999.97,0.044358
36,TMO,618.72,2026-01-16,2026-02-13,605.48,665.74,663.85,0.042512
4,AMZN,239.12,2026-01-16,2026-02-13,231.47,261.29,254.30,0.041402
6,BA,247.68,2026-01-16,2026-02-13,233.76,271.09,268.00,0.040119
23,ISRG,535.00,2026-01-16,2026-02-13,540.50,553.79,572.10,0.038249
14,DE,514.40,2026-01-16,2026-02-13,499.93,574.11,523.50,0.035213
40,UPS,106.91,2026-01-16,2026-02-13,102.70,117.77,111.23,0.034203
33,PYPL,56.89,2026-01-16,2026-02-13,56.00,59.16,61.06,0.032519


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
df9 = run_model('cry', '11b', tickers9, False)

In [ ]:
addAllocHL(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

In [ ]:
run_model_nn_cmd('country', '1112').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
3,ECH,42.67,2026-01-06,2026-02-03,39.97,43.68,40.05,-0.033669
21,THD,60.99,2026-01-06,2026-02-03,57.16,61.53,58.40,-0.032136
9,EWC,54.46,2026-01-06,2026-02-03,51.37,55.35,52.63,-0.024666
11,EWP,54.85,2026-01-06,2026-02-03,52.06,55.46,53.26,-0.022911
19,KEMQ,26.36,2026-01-06,2026-02-03,25.83,27.77,27.24,0.022256
2,FLN,23.60,2026-01-06,2026-02-03,22.50,24.05,23.03,-0.017232
5,EWA,26.42,2026-01-06,2026-02-03,25.11,27.04,25.78,-0.016780
7,EWI,55.37,2026-01-06,2026-02-03,52.59,56.02,54.91,-0.015592
23,VNM,19.35,2026-01-06,2026-02-03,18.40,19.92,18.87,-0.014815
4,EPHE,25.94,2026-01-06,2026-02-03,25.31,27.36,26.26,0.014264


In [ ]:
run_model_nn_cmd('global', '1107').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
65,VNT,38.38,2026-01-05,2026-02-02,38.40,43.58,42.70,0.082856
37,LIN,430.69,2026-01-05,2026-02-02,414.06,482.51,501.75,0.082232
9,COTY,3.13,2026-01-05,2026-02-02,3.03,3.68,3.45,0.082002
59,TEF,3.99,2026-01-05,2026-02-02,3.78,4.50,4.56,0.072682
2,ADYEY,16.89,2026-01-05,2026-02-02,16.45,18.56,19.01,0.066114
31,IFNNy,46.80,2026-01-05,2026-02-02,39.59,48.72,43.62,-0.060328
49,POAHF,46.50,2026-01-02,2026-01-30,45.36,50.17,52.19,0.058925
7,CAAP,26.85,2026-01-05,2026-02-02,23.58,27.73,24.71,-0.056238
57,SIEGY,143.00,2026-01-05,2026-02-02,138.71,156.34,156.86,0.053403
12,CNI,99.75,2026-01-05,2026-02-02,96.92,110.96,106.05,0.049056


In [ ]:
run_model_nn_cmd('valuestock', '0301').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
17,CCI,87.72,2026-01-06,2026-02-03,59.37,87.81,76.06,-0.151695
10,BAX,20.41,2026-01-06,2026-02-03,19.22,24.97,23.22,0.100931
76,SAFE,14.13,2026-01-06,2026-02-03,13.77,16.23,16.61,0.099552
48,HPE,23.91,2026-01-06,2026-02-03,23.81,24.98,28.01,0.070682
78,SCCO,160.62,2026-01-06,2026-02-03,139.13,164.56,147.48,-0.063691
55,KMB,97.88,2026-01-06,2026-02-03,91.76,114.36,103.16,0.053262
50,IP,40.88,2026-01-06,2026-02-03,38.89,46.67,43.59,0.053082
51,IPI,28.43,2026-01-06,2026-02-03,27.92,31.39,30.47,0.052644
91,URI,893.78,2026-01-06,2026-02-03,892.12,974.29,951.47,0.050922
49,IFF,68.47,2026-01-06,2026-02-03,67.42,74.57,73.82,0.050630


In [ ]:
run_model_nn_cmd('commodity', '0312').head(5)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
12,USO,71.59,2026-01-09,2026-02-06,69.79,74.33,72.23,0.007357
3,FXF,110.36,2026-01-09,2026-02-06,107.57,114.37,111.27,0.006433
4,FXY,58.20,2026-01-09,2026-02-06,56.91,60.10,58.71,0.006415
15,WEAT,20.32,2026-01-09,2026-02-06,19.83,21.11,20.31,0.004757
14,VTIP,49.51,2026-01-09,2026-02-06,48.23,50.75,49.08,-0.003164


In [ ]:
run_model_nn_cmd('crypto', '0214').head(5)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
5,DOT-USD,2.11,2026-01-09,2026-02-06,2.00,2.77,2.14,0.091627
1,ETH-USD,3119.00,2026-01-09,2026-02-06,2663.54,3318.13,2727.55,-0.069229
11,ICP-USD,3.34,2026-01-09,2026-02-06,3.21,4.20,3.30,0.068862
18,ATOM-USD,2.52,2026-01-09,2026-02-06,2.45,3.27,2.32,0.063492
17,MATIC-USD,0.22,2025-03-24,2025-04-21,0.19,0.26,0.17,-0.060606


In [ ]:
run_model_nn_cmd('techstock', '0312').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
30,GTLB,35.88,2026-01-09,2026-02-06,36.85,45.05,44.10,0.170569
80,VEEV,241.63,2026-01-09,2026-02-06,246.03,293.95,287.29,0.141235
77,UI,551.52,2026-01-09,2026-02-06,460.33,549.41,438.86,-0.124480
62,RBLX,72.60,2026-01-09,2026-02-06,61.19,73.00,58.75,-0.114141
52,NTNX,53.66,2026-01-09,2026-02-06,53.44,68.01,55.36,0.098335
66,SNOW,219.94,2026-01-09,2026-02-06,218.33,247.38,257.87,0.096632
20,DT,42.68,2026-01-09,2026-02-06,43.00,48.26,48.40,0.090753
76,U,45.38,2026-01-09,2026-02-06,36.66,47.57,39.59,-0.090495
17,DOCN,51.92,2026-01-09,2026-02-06,44.15,53.89,44.67,-0.083783
39,JKS,27.39,2026-01-09,2026-02-06,26.02,31.81,31.10,0.082268


In [ ]:
run_model_nn('stock', '0123').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
204,PUBM,12.82,2025-07-03,2025-07-31,12.64,16.44,14.37,0.129745
375,PI,115.86,2025-07-03,2025-07-31,116.02,144.76,128.19,0.119080
17,INTA,48.11,2025-07-03,2025-07-31,45.99,55.07,59.27,0.110857
352,ABR,10.90,2025-07-03,2025-07-31,10.53,12.98,12.60,0.104281
144,LFMD,12.50,2025-07-03,2025-07-31,11.93,14.68,14.55,0.097600
378,RCAT,7.08,2025-07-03,2025-07-31,5.73,7.56,5.93,-0.095104
10,ARQT,13.82,2025-07-03,2025-07-31,13.37,16.73,15.27,0.094308
348,CRM,272.15,2025-07-03,2025-07-31,267.03,319.47,305.57,0.092620
283,NTGR,29.41,2025-07-03,2025-07-31,28.97,35.49,31.90,0.092146
150,POWL,217.66,2025-07-03,2025-07-31,204.27,250.88,256.42,0.089727


In [ ]:
run_model_nn_cmd('buystocks', '0402', False, 500, 250).head(10)
# show_prediction('buystocks', '0508').head(10)

In [ ]:
run_model_nn_cmd('bluechipstock', '0117', False, 500, 250).head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
5,AVGO,351.71,2026-01-16,2027-01-01,302.49,647.68,625.19,0.493048
32,ORCL,191.09,2026-01-16,2027-01-01,179.33,372.10,285.73,0.460324
21,INTC,46.96,2026-01-16,2027-01-01,42.58,82.98,78.60,0.449177
17,DIS,111.20,2026-01-16,2027-01-01,109.17,177.38,161.59,0.343345
29,NKE,64.38,2026-01-16,2027-01-01,60.36,98.52,99.34,0.336958
6,BA,247.68,2026-01-16,2027-01-01,238.14,371.94,364.78,0.311989
22,IBM,305.67,2026-01-16,2027-01-01,294.48,450.16,409.01,0.258056
18,GOOGL,330.00,2026-01-16,2027-01-01,302.62,487.73,436.08,0.238818
39,UNH,331.02,2026-01-16,2027-01-01,180.86,406.53,222.80,-0.184148
13,CSCO,75.19,2026-01-16,2027-01-01,68.31,103.91,93.86,0.179589


In [9]:
run_model_nn_cmd('pennystock', '0122', False, 500, 250).head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
38,ONDS,12.55,2026-01-21,2027-01-06,9.32,24.94,23.65,0.538114
83,RUN,17.82,2026-01-21,2027-01-06,16.38,35.13,30.49,0.533857
101,IHS,7.68,2026-01-21,2027-01-06,6.50,15.12,13.60,0.528646
44,PGY,21.56,2026-01-21,2027-01-06,18.30,42.49,37.51,0.519790
86,SMR,19.75,2026-01-21,2027-01-06,12.70,38.35,37.51,0.494684
9,BGSF,5.62,2026-01-21,2027-01-06,5.19,10.81,8.67,0.463227
98,GPUS,0.28,2026-01-21,2027-01-06,0.05,0.36,0.05,-0.452381
36,NEPH,4.56,2026-01-21,2027-01-06,3.85,8.87,7.13,0.451023
62,VSAT,42.64,2026-01-21,2027-01-06,35.88,81.16,68.12,0.447467
74,EOSE,17.53,2026-01-21,2027-01-06,11.49,31.87,32.30,0.438677


In [ ]:
import sys
sys.path.append('/content')
!unzip drive/MyDrive/colab/stock_prediction_r.zip
import stock_prediction.stock_prediction.stock_predictor_lib
from stock_prediction.stock_prediction.stock_predictor_lib import run_model_nn, show_prediction, StockPredictorLib, Config
colab_path = 'content/drive/MyDrive/colab'

In [ ]:
techcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
colab_path = 'content/drive/MyDrive/colab'
techcfg.model_path = f'/{colab_path}/models/transformer/techstock_1220'
techdf = run_model_nn(f'/{colab_path}/tickers/techstocktickers.csv', cfg=techcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/techstock_1220_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/techstock_1220_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/techstock_1220_close_model.pth'}


In [ ]:
techdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
26,EXPE,289.25,2025-12-19,2026-01-16,189.62,291.44,189.38,-0.227381
48,MU,265.92,2025-12-19,2026-01-16,174.66,270.49,171.69,-0.226785
11,COHR,185.83,2025-12-19,2026-01-16,108.07,191.14,134.34,-0.222318
5,APP,721.37,2025-12-19,2026-01-16,553.35,728.21,491.45,-0.180721
77,UI,564.62,2025-12-19,2026-01-16,388.92,595.41,428.44,-0.165946
51,NVCR,12.84,2025-12-19,2026-01-16,11.25,16.00,16.78,0.143043
73,TRIP,13.87,2025-12-19,2026-01-16,12.95,16.06,18.45,0.140591
33,HOOD,121.35,2025-12-19,2026-01-16,92.11,122.82,99.67,-0.135833
42,MBLY,10.19,2025-12-19,2026-01-16,9.87,12.39,11.95,0.119071
72,TOST,36.22,2025-12-19,2026-01-16,29.83,36.82,30.22,-0.108504


In [ ]:
techrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
techrcfg.model_path = f'/{colab_path}/models/rolling/techstock_0109'
techrdf = run_model_nn(f'/{colab_path}/tickers/techstocktickers.csv', cfg=techrcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/techstock_0109_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/techstock_0109_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/techstock_0109_close_model.pth'}


In [ ]:
techrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
37,IOT,33.68,2026-01-09,2026-02-06,26.74,34.82,29.64,-0.097387
75,TWLO,133.71,2026-01-09,2026-02-06,131.95,151.35,150.98,0.082642
73,TRIP,14.59,2026-01-09,2026-02-06,11.65,16.43,12.12,-0.081563
61,PUBM,8.38,2026-01-09,2026-02-06,7.82,10.64,8.64,0.077963
11,COHR,173.30,2026-01-09,2026-02-06,157.94,202.59,199.70,0.077573
7,BMBL,3.48,2026-01-09,2026-02-06,3.42,4.07,3.60,0.062261
2,AI,14.10,2026-01-09,2026-02-06,11.68,16.06,12.00,-0.060520
23,ENPH,34.94,2026-01-09,2026-02-06,30.78,35.89,31.90,-0.059626
15,DBX,27.30,2026-01-09,2026-02-06,26.36,30.48,29.93,0.059463
35,INMD,14.31,2026-01-09,2026-02-06,13.78,15.51,16.14,0.058234


In [ ]:
defetfcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
defetfcfg.model_path = f'/{colab_path}/models/rolling/etf_1221'
etfdf = run_model_nn(f'/{colab_path}/tickers/etftickers.csv', cfg=defetfcfg)
etfdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/etf_1221_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/etf_1221_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/etf_1221_close_model.pth'}


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
4,DXYZ,31.12,2025-12-23,2026-01-20,30.03,33.07,34.27,0.042952
17,SMH,362.57,2025-12-23,2026-01-20,350.09,383.28,387.92,0.030872
32,XSW,190.69,2025-12-23,2026-01-20,183.57,201.97,203.57,0.029787
26,XLK,145.59,2025-12-23,2026-01-20,140.50,153.66,155.37,0.029214
0,ARKF,48.98,2025-12-23,2026-01-20,47.21,51.84,52.02,0.028107
2,ARKW,154.10,2025-12-23,2026-01-20,148.54,162.98,163.73,0.028012
16,QQQ,621.32,2025-12-23,2026-01-20,599.29,655.97,660.43,0.027753
18,SPY,687.58,2025-12-23,2026-01-20,663.48,725.22,728.49,0.026397
7,IPO,46.94,2025-12-23,2026-01-20,45.28,49.61,49.59,0.025991
5,EEM,54.24,2025-12-23,2026-01-20,52.32,57.21,57.41,0.025934


In [ ]:
tvolcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
tvolcfg.model_path = f'/{colab_path}/models/transformer/pennystock_1226'
tvoldf = run_model_nn(f'/{colab_path}/tickers/pennystocktickers.csv', cfg=tvolcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/pennystock_1226_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/pennystock_1226_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/pennystock_1226_close_model.pth'}


In [ ]:
tvoldf[tvoldf['predicted_return']> 0].head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
6,ATOM,2.18,2025-12-26,2026-01-23,2.14,3.61,2.33,0.235474
67,YEXT,8.05,2025-12-26,2026-01-23,7.86,11.77,8.65,0.171014
58,SLE,0.62,2025-12-26,2026-01-23,0.52,0.91,0.74,0.166667
31,IRDM,17.17,2025-12-26,2026-01-23,16.32,20.08,23.59,0.164628
63,TDOC,7.18,2025-12-26,2026-01-23,6.49,10.70,7.46,0.144383
40,NNOX,2.85,2025-12-26,2026-01-23,2.66,4.12,3.00,0.143860
61,STEM,16.19,2025-12-26,2026-01-23,13.66,28.20,13.48,0.139386
49,PLUG,2.10,2025-12-26,2026-01-23,1.86,3.07,2.22,0.134921
30,INVZ,0.95,2025-12-26,2026-01-23,0.90,1.34,0.99,0.133333
54,RVYL,0.18,2025-12-26,2026-01-23,0.16,0.28,0.17,0.129630


In [ ]:
tvolrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
tvolrcfg.model_path = f'/{colab_path}/models/rolling/pennystock_0114'
tvolrdf = run_model_nn(f'/{colab_path}/tickers/pennystocktickers.csv', cfg=tvolrcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/pennystock_0114_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/pennystock_0114_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/pennystock_0114_close_model.pth'}


In [ ]:
tvolrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
82,WKEY,8.72,2026-01-14,2026-02-11,6.54,13.68,10.52,0.175076
39,INVZ,1.06,2026-01-14,2026-02-11,0.93,1.55,1.22,0.163522
32,GPUS,0.32,2026-01-14,2026-02-11,0.24,0.34,0.25,-0.135417
48,NEPH,4.40,2026-01-14,2026-02-11,4.14,5.58,5.25,0.134091
15,CARS,11.97,2026-01-14,2026-02-11,11.23,14.27,14.62,0.117238
9,BBAI,6.12,2026-01-14,2026-02-11,4.24,6.48,5.78,-0.101307
79,UDMY,5.14,2026-01-14,2026-02-11,4.83,6.15,5.99,0.100519
81,VSAT,42.03,2026-01-14,2026-02-11,33.30,44.14,36.34,-0.097629
30,GRAB,4.65,2026-01-14,2026-02-11,4.53,5.25,5.46,0.092473
68,SFIX,5.24,2026-01-14,2026-02-11,4.01,5.67,4.65,-0.088422


In [ ]:
bctcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
bctcfg.model_path = f'/{colab_path}/models/transformer/bluechipstock_1010'
bctdf = run_model_nn(f'/{colab_path}/tickers/bluechipstocktickers.csv', cfg=bctcfg)
bctdf.head(10)

In [ ]:
bcrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
bcrcfg.model_path = f'/{colab_path}/models/rolling/bluechipstock_0117'
bcrdf = run_model_nn(f'/{colab_path}/tickers/bluechipstocktickers.csv', cfg=bcrcfg)
bcrdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/bluechipstock_0117_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/bluechipstock_0117_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/bluechipstock_0117_close_model.pth'}


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
29,NKE,64.38,2026-01-16,2026-02-13,62.87,74.50,69.46,0.070881
16,DHR,235.99,2026-01-16,2026-02-13,230.89,256.36,258.42,0.053251
25,LLY,1038.40,2026-01-16,2026-02-13,999.85,1136.99,1135.79,0.050536
17,DIS,111.20,2026-01-16,2026-02-13,108.60,120.33,118.70,0.042056
11,CRM,227.11,2026-01-16,2026-02-13,220.41,251.22,236.99,0.040054
12,COST,963.61,2026-01-16,2026-02-13,923.90,1031.77,1050.03,0.039736
31,NOW,127.31,2026-01-16,2026-02-13,121.23,142.72,133.10,0.039588
0,AAPL,255.53,2026-01-16,2026-02-13,253.88,283.29,259.19,0.038834
2,ADBE,296.12,2026-01-16,2026-02-13,284.27,337.85,298.30,0.036089
7,BKNG,5115.91,2026-01-16,2026-02-13,4977.34,5434.66,5410.26,0.030919


In [ ]:
buystkcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
buystkcfg.model_path = f'/{colab_path}/models/transformer/buystocks_1115'
buystkcfg.model.window_size = 500
buystkcfg.model.future_days = 250
buystkdf = run_model_nn(f'/{colab_path}/tickers/buystockstickers.csv', cfg=buystkcfg)

In [ ]:
buystkdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
130,LFMD,4.63,2025-11-14,2026-10-30,3.63,11.99,64.18,4.745140
356,LAES,4.49,2025-11-14,2026-10-30,3.21,39.63,30.40,4.437268
327,OKLO,97.57,2025-11-14,2026-10-30,82.54,984.50,406.78,4.035086
315,INOD,57.21,2025-11-14,2026-10-30,39.89,373.78,401.43,3.749170
345,NUTX,107.11,2025-11-14,2026-10-30,100.94,336.41,954.96,3.332960
95,CHGG,1.09,2025-11-14,2026-10-30,0.58,4.85,8.61,3.293578
329,RGTI,25.48,2025-11-14,2026-10-30,19.92,215.40,82.17,3.153454
330,ACHR,7.88,2025-11-14,2026-10-30,7.71,64.69,23.58,3.060068
258,WKEY,9.20,2025-11-14,2026-10-30,7.15,37.14,65.95,2.994203
352,BULL,8.63,2025-11-14,2026-10-30,4.07,57.04,40.67,2.931248


In [ ]:
buystkrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
buystkrcfg.model_path = f'/{colab_path}/models/rolling/buystocks_1213'
buystkrcfg.model.window_size = 500
buystkrcfg.model.future_days = 250
buystkrdf = run_model_nn(f'/{colab_path}/tickers/buystockstickers.csv', cfg=buystkrcfg)

In [ ]:
buystkrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
159,LAZR,0.90,2025-12-12,2026-11-27,0.78,4.36,2.42,1.800000
160,NOW,865.06,2025-12-12,2026-11-27,743.05,3287.56,1157.09,0.998975
358,SQNS,5.95,2025-12-12,2026-11-27,5.34,19.88,9.82,0.963025
246,COSM,0.44,2025-12-12,2026-11-27,0.40,1.05,0.78,0.689394
5,DOMO,9.04,2025-12-12,2026-11-27,8.56,15.78,21.07,0.674410
35,UP,0.70,2025-12-12,2026-11-27,0.61,1.44,1.45,0.666667
349,BULL,9.16,2025-12-12,2026-11-27,8.34,26.16,9.34,0.595342
23,MBLY,11.07,2025-12-12,2026-11-27,9.79,23.06,19.97,0.590485
130,LFMD,3.40,2025-12-12,2026-11-27,3.35,6.58,6.11,0.572549
366,QUBT,12.05,2025-12-12,2026-11-27,8.94,22.31,24.24,0.534993


In [ ]:
stkcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
stkcfg.model_path = f'/{colab_path}/models/transformer/stock_0105'
stkdf = run_model_nn(f'/{colab_path}/tickers/stocktickers.csv', cfg=stkcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/stock_0105_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/stock_0105_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/stock_0105_close_model.pth'}


In [ ]:
stkdf[stkdf['predicted_return'] > 0].head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
56,LCID,11.52,2026-01-21,2026-02-18,10.80,31.75,19.45,0.793981
36,UP,0.89,2026-01-21,2026-02-18,0.75,1.85,0.74,0.250936
27,INSG,11.89,2026-01-21,2026-02-18,11.02,16.57,13.73,0.158396
340,AEVA,17.93,2026-01-21,2026-02-18,16.89,22.60,21.69,0.137386
260,BYRN,15.89,2026-01-21,2026-02-18,14.69,20.71,18.74,0.135725
348,PATH,14.23,2026-01-21,2026-02-18,13.35,16.34,18.48,0.128367
231,PAGS,10.49,2026-01-21,2026-02-18,9.91,13.88,11.48,0.120750
98,SLP,20.59,2026-01-21,2026-02-18,19.87,23.66,24.75,0.105391
38,NXDR,1.94,2026-01-21,2026-02-18,1.79,2.58,2.04,0.101375
236,SFM,71.71,2026-01-21,2026-02-18,67.29,84.78,84.47,0.099521


In [ ]:
stkrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
stkrcfg.model_path = f'/{colab_path}/models/rolling/stock_0102'
stkrdf = run_model_nn(f'/{colab_path}/tickers/stocktickers.csv', cfg=stkrcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/stock_0102_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/stock_0102_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/stock_0102_close_model.pth'}


In [ ]:
stkrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
371,SQNS,5.39,2026-01-20,2026-02-17,5.07,7.97,6.17,0.188002
217,LEU,320.85,2026-01-20,2026-02-17,293.17,468.52,370.09,0.175814
84,SKYT,35.59,2026-01-20,2026-02-17,31.63,44.87,45.52,0.142830
340,AEVA,19.38,2026-01-20,2026-02-17,13.12,21.39,15.46,-0.140523
386,ADA-USD,0.36,2026-01-20,2026-02-17,0.35,0.44,0.43,0.129630
320,INOD,64.01,2026-01-20,2026-02-17,61.77,84.21,69.82,0.123783
334,MU,373.92,2026-01-20,2026-02-17,350.32,447.51,449.28,0.111744
216,RDDT,227.66,2026-01-20,2026-02-17,219.63,267.49,271.58,0.110867
378,CDE,22.95,2026-01-20,2026-02-17,18.59,25.64,17.37,-0.105301
81,NBIS,102.19,2026-01-20,2026-02-17,91.52,125.35,120.31,0.099847


In [ ]:
glbcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
glbcfg.model_path = f'/{colab_path}/models/transformer/global_0105'
glbdf = run_model_nn(f'globaltickers.csv', cfg=glbcfg)


In [ ]:
glbdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
63,TSM,328.76,2026-01-05,2026-02-02,225.74,328.58,259.19,-0.175173
61,TKAMY,11.70,2026-01-05,2026-02-02,8.24,11.68,9.85,-0.151852
54,SFTBY,60.25,2026-01-05,2026-02-02,47.08,60.75,46.01,-0.148880
33,KNNGF,66.45,2026-01-02,2026-01-30,50.24,67.80,51.88,-0.147630
3,BAK,2.85,2026-01-05,2026-02-02,2.73,3.64,3.23,0.122807
6,BESIY,178.00,2026-01-05,2026-02-02,151.29,175.31,149.17,-0.109045
55,SAP,241.76,2026-01-05,2026-02-02,238.29,289.49,273.81,0.105215
50,PROSF,60.00,2026-01-02,2026-01-30,56.19,73.90,67.46,0.097500
31,IFNNy,46.82,2026-01-05,2026-02-02,41.21,46.83,39.98,-0.088566
24,FANUY,20.34,2026-01-05,2026-02-02,16.34,20.54,18.83,-0.087021


In [ ]:
glbrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
glbrcfg.model_path = f'/{colab_path}/models/rolling/global_1210'
glbrdf = run_model_nn(f'/{colab_path}/tickers/globaltickers.csv', cfg=glbrcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/global_1210_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/global_1210_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/global_1210_close_model.pth'}


In [ ]:
glbrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
34,KNNGF,66.45,2025-10-23,2025-11-20,54.51,66.45,58.78,-0.098370
57,SE,124.22,2025-12-10,2026-01-07,120.04,137.44,150.25,0.094107
37,RACE,364.04,2025-12-10,2026-01-07,364.08,407.93,406.34,0.078957
51,PROSF,60.00,2025-12-08,2026-01-05,56.53,69.72,66.23,0.069333
24,EVKIY,7.54,2025-12-09,2026-01-06,7.39,8.82,7.57,0.051282
35,MELI,1970.73,2025-12-10,2026-01-07,1956.95,2137.48,2095.23,0.046932
23,ESEA,59.00,2025-12-10,2026-01-07,56.62,63.81,64.52,0.044915
56,SAP,245.88,2025-12-10,2026-01-07,236.16,263.98,269.01,0.042717
61,TGLS,53.41,2025-12-10,2026-01-07,45.93,59.29,48.65,-0.039693
18,ENGGY,7.96,2025-12-10,2026-01-07,7.70,8.42,8.61,0.035595


In [ ]:
ctcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
ctcfg.model_path = f'/{colab_path}/models/transformer/country_1211'
ctdf = run_model_nn(f'/{colab_path}/tickers/countrytickers.csv', cfg=ctcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/country_1211_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/country_1211_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/country_1211_close_model.pth'}


In [ ]:
ctdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
3,ECH,39.48,2025-12-11,2026-01-08,33.26,39.97,35.66,-0.080632
0,ARGT,91.19,2025-12-11,2026-01-08,90.88,101.31,100.06,0.068282
11,EWP,53.54,2025-12-11,2026-01-08,49.38,54.03,49.98,-0.045013
5,EWA,26.71,2025-12-11,2026-01-08,25.28,26.69,24.93,-0.040309
1,EWD,49.70,2025-12-11,2026-01-08,47.42,49.82,46.30,-0.037290
16,IDX,17.23,2025-12-11,2026-01-08,15.67,17.30,16.81,-0.036951
6,EWG,42.46,2025-12-11,2026-01-08,39.70,42.48,40.86,-0.034071
14,EWY,95.07,2025-12-11,2026-01-08,84.34,94.61,96.79,-0.033204
13,EWW,71.42,2025-12-11,2026-01-08,70.08,71.49,68.28,-0.020582
2,FLN,22.91,2025-12-11,2026-01-08,21.06,23.02,23.28,-0.019933


In [ ]:
valtcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
valtcfg.model_path = f'/{colab_path}/models/transformer/valuestock_1018'
valtdf = run_model_nn(f'valuestocktickers.csv', cfg=valtcfg)


In [ ]:
valtdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
71,POWL,355.91,2025-11-12,2025-12-10,318.63,406.16,497.69,0.144934
16,CC,12.51,2025-11-12,2025-12-10,11.58,15.90,14.90,0.129230
7,ARE,53.86,2025-11-12,2025-12-10,52.99,59.89,66.73,0.111586
91,UAL,99.94,2025-11-12,2025-12-10,91.43,119.48,121.23,0.107798
14,BXC,57.66,2025-11-12,2025-12-10,55.51,69.67,63.39,0.090126
44,GLW,88.71,2025-11-12,2025-12-10,86.23,108.45,94.80,0.087739
74,RH,160.72,2025-11-12,2025-12-10,156.87,193.12,172.52,0.083686
37,F,13.45,2025-11-12,2025-12-10,12.72,13.75,10.53,-0.083024
58,MOD,147.99,2025-11-12,2025-12-10,142.12,179.25,154.26,0.071311
39,FICO,1778.19,2025-11-12,2025-12-10,1648.44,2256.72,1784.05,0.066480


In [ ]:
valrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
valrcfg.model_path = f'/{colab_path}/models/rolling/valuestock_0106'
valrdf = run_model_nn(f'/{colab_path}/tickers/valuestocktickers.csv', cfg=valrcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/valuestock_0106_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/valuestock_0106_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/valuestock_0106_close_model.pth'}


In [ ]:
valrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
58,MOD,128.36,2026-01-06,2026-02-03,125.35,155.17,148.67,0.114548
67,PETS,3.53,2026-01-06,2026-02-03,2.93,4.71,3.73,0.073654
72,PSTL,15.95,2026-01-06,2026-02-03,15.59,17.75,17.66,0.065831
89,TWI,8.28,2026-01-06,2026-02-03,7.38,8.56,7.59,-0.052738
95,WHR,78.12,2026-01-06,2026-02-03,67.28,80.95,74.14,-0.051161
86,TGT,103.83,2026-01-06,2026-02-03,91.59,109.68,94.91,-0.049151
18,CCJ,102.04,2026-01-06,2026-02-03,94.15,103.45,93.87,-0.047857
94,WDC,217.23,2026-01-06,2026-02-03,183.11,227.98,209.55,-0.047645
35,EQT,52.58,2026-01-06,2026-02-03,50.01,59.68,55.51,0.047293
2,ALB,160.02,2026-01-06,2026-02-03,134.73,166.33,156.60,-0.046661


In [ ]:
commtcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
commtcfg.model_path = f'/{colab_path}/models/transformer/commodity_0109'
commtdf = run_model_nn(f'/{colab_path}/tickers/commoditytickers.csv', cfg=commtcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/commodity_0109_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/commodity_0109_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/commodity_0109_close_model.pth'}


In [ ]:
commtdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
8,SLV,72.64,2026-01-09,2026-02-06,64.95,71.24,58.03,-0.108756
4,FXY,58.19,2026-01-09,2026-02-06,54.04,58.52,55.80,-0.035573
7,PALL,167.62,2026-01-09,2026-02-06,143.90,183.02,163.57,-0.024599
9,SOYB,22.22,2026-01-09,2026-02-06,21.71,22.48,21.47,-0.015002
12,USO,70.54,2026-01-08,2026-02-05,68.92,74.53,70.86,0.012711
5,GLD,414.94,2026-01-09,2026-02-06,405.02,417.61,409.34,-0.010323
14,VTIP,49.52,2026-01-09,2026-02-06,49.22,50.79,49.91,0.009155
11,TLT,87.74,2026-01-09,2026-02-06,87.15,88.98,89.30,0.008396
15,WEAT,20.32,2026-01-09,2026-02-06,20.12,20.95,19.55,-0.005577
0,DBC,23.05,2026-01-09,2026-02-06,22.29,23.55,22.96,-0.005061


In [ ]:
cryptcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
cryptcfg.model_path = f'/{colab_path}/models/transformer/crypto_1021'
cryptdf = run_model_nn(f'/{colab_path}/tickers/cryptotickers.csv', cfg=cryptcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_close_model.pth'}


In [ ]:
cryptdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
15,XLM-USD,0.23,2025-11-21,2025-12-19,0.22,0.28,0.29,0.144928
6,DOGE-USD,0.14,2025-11-21,2025-12-19,0.14,0.16,0.18,0.142857
3,ADA-USD,0.41,2025-11-21,2025-12-19,0.39,0.48,0.53,0.138211
4,XRP-USD,1.96,2025-11-21,2025-12-19,1.93,2.34,2.39,0.132653
5,DOT-USD,2.37,2025-11-21,2025-12-19,2.25,3.10,2.64,0.123769
11,ICP-USD,4.36,2025-11-21,2025-12-19,3.64,4.84,5.74,0.087156
12,AVAX-USD,13.32,2025-11-21,2025-12-19,8.87,16.84,11.20,-0.076326
17,MATIC-USD,0.22,2025-03-24,2025-04-21,0.15,0.26,0.20,-0.075758
18,ATOM-USD,2.53,2025-11-21,2025-12-19,2.31,3.29,2.50,0.067194
16,AAVE-USD,157.80,2025-11-21,2025-12-19,152.88,185.84,162.92,0.059654


In [ ]:
cryprcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
cryprcfg.model_path = f'/{colab_path}/models/rolling/crypto_0109'
cryprdf = run_model_nn(f'/{colab_path}/tickers/cryptotickers.csv', cfg=cryprcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/crypto_0109_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/crypto_0109_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/crypto_0109_close_model.pth'}


In [ ]:
cryprdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
4,XRP-USD,2.13,2026-01-09,2026-02-06,1.79,3.05,2.55,0.156495
16,AAVE-USD,168.97,2026-01-09,2026-02-06,158.12,208.42,179.54,0.077272
17,MATIC-USD,0.22,2025-03-24,2025-04-21,0.19,0.24,0.19,-0.060606
11,ICP-USD,3.34,2026-01-09,2026-02-06,2.92,3.62,2.88,-0.059880
5,DOT-USD,2.12,2026-01-09,2026-02-06,2.01,2.41,2.27,0.051887
6,DOGE-USD,0.14,2026-01-09,2026-02-06,0.12,0.15,0.13,-0.047619
8,LTC-USD,81.53,2026-01-09,2026-02-06,75.82,94.60,85.09,0.044646
15,XLM-USD,0.23,2026-01-09,2026-02-06,0.22,0.25,0.25,0.043478
2,SOL-USD,139.11,2026-01-09,2026-02-06,130.26,159.47,114.70,-0.030911
1,ETH-USD,3120.14,2026-01-09,2026-02-06,3100.30,3394.84,2603.69,-0.027946


In [ ]:
etftcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
etftcfg.model_path = f'/{colab_path}/models/transformer/etf_0112'
etftdf = run_model_nn(f'/{colab_path}/tickers/etftickers.csv', cfg=etftcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/etf_0112_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/etf_0112_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/etf_0112_close_model.pth'}


In [ ]:
etftdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
19,URA,51.76,2026-01-12,2026-02-09,41.08,52.38,41.30,-0.132148
4,DXYZ,30.37,2026-01-12,2026-02-09,29.50,34.27,38.67,0.124355
31,XME,120.71,2026-01-12,2026-02-09,108.28,122.34,93.18,-0.105846
7,IPO,48.51,2026-01-12,2026-02-09,44.65,48.64,43.78,-0.058132
12,IWO,344.97,2026-01-12,2026-02-09,310.29,351.61,317.84,-0.053309
23,XLE,46.38,2026-01-12,2026-02-09,46.01,49.96,48.83,0.040678
13,IYZ,33.79,2026-01-12,2026-02-09,30.00,35.06,32.47,-0.037881
14,JETS,29.26,2026-01-12,2026-02-09,26.47,29.53,29.07,-0.030873
30,XLY,124.51,2026-01-12,2026-02-09,117.95,128.40,116.84,-0.027682
18,SPY,695.61,2026-01-12,2026-02-09,690.33,729.39,712.75,0.021870


In [ ]:
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
from google.auth import default


auth.authenticate_user()


creds, _ = default()
gc = gspread.authorize(creds)
# gc = gspread.authorize(GoogleCredentials.get_application_default())


In [ ]:
# Open your spreadsheet by name
spreadsheet = gc.open('Allocation')
# Or by URL: spreadsheet = gc.open_by_url('Your Spreadsheet URL')

# Select the worksheet (e.g., the first sheet)
worksheet = spreadsheet.worksheet("Buy")

# Get all values as a list of lists
data = worksheet.get_all_values()

# Convert to a Pandas DataFrame
sheetdf = pd.DataFrame(data[1:], columns=data[0]) # Assuming first row is header


In [ ]:
df = sheetdf[['Ticker']].copy()

# Apply the extraction function first, then filter based on the result
def extract_ticker(ticker_str):
    # Convert to string before splitting
    ticker_str = str(ticker_str)
    parts = ticker_str.split(':')
    if len(parts) > 1:
        return parts[1].strip() # Use strip to remove leading/trailing whitespace
    return ticker_str.strip() # Return the original string if no colon, also strip whitespace

df['Ticker'] = df['Ticker'].apply(extract_ticker)
# Remove BTCUSD and add BTC-USD, ETH-USD, ADA-USD, SOL-USD in Ticker
df = df[df['Ticker'] != 'BTCUSD']
df = pd.concat([df, pd.DataFrame({'Ticker': ['BTC-USD', 'ETH-USD', 'ADA-USD', 'SOL-USD']})], ignore_index=True)
df.to_csv(f'/{colab_path}/tickers/stocktickers.csv', index=False)
df.to_csv(f'/{colab_path}/tickers/buystockstickers.csv', index=False)

In [ ]:
# Only have Tickers column

run_model_nn('buystocks', '0402', False, 500, 250, copy_tickers=False).head(10)


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
333,QBTS,34.25,2025-10-08,2026-09-23,32.44,71.01,65.57,0.644964
96,CHGG,1.42,2025-10-08,2026-09-23,1.40,2.82,2.74,0.633803
97,SLP,16.47,2025-10-08,2026-09-23,16.09,32.07,31.69,0.616070
18,OUST,30.65,2025-10-08,2026-09-23,29.45,61.59,56.80,0.607830
9,HIMS,58.17,2025-10-08,2026-09-23,49.10,110.63,110.40,0.547934
252,LYFT,21.16,2025-10-08,2026-09-23,19.66,41.40,36.93,0.543636
6,DOMO,14.74,2025-10-08,2026-09-23,12.98,29.59,25.17,0.531886
326,ROOT,81.73,2025-10-08,2026-09-23,76.40,156.09,141.76,0.526367
122,LZ,9.97,2025-10-08,2026-09-23,9.63,19.32,16.65,0.524574
276,CRNC,11.96,2025-10-08,2026-09-23,11.62,21.43,21.43,0.518395


In [ ]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +180 -delete -print

sending incremental file list
buystocks_0709_predictions.csv

sent 18,897 bytes  received 35 bytes  37,864.00 bytes/sec
total size is 18,770  speedup is 0.99
